In [ ]:
print "Uploading Libraries"
import arcpy
import pandas as pd
import datetime
import numpy as np
arcpy.overwriteoutput = True
arcpy.env.overwriteOutput = True

def table_to_data_frame(in_table, input_fields=None, where_clause=None):
    """Function will convert an arcgis table into a pandas dataframe with an object ID index, and the selected
    input fields using an arcpy.da.SearchCursor."""
    OIDFieldName = arcpy.Describe(in_table).OIDFieldName
    # print OIDFieldName
    if input_fields:
        final_fields = [OIDFieldName] + input_fields + ['SHAPE@JSON']
    else:
        final_fields = [field.name for field in arcpy.ListFields(in_table)] + ['SHAPE@JSON']
    data = [row for row in arcpy.da.SearchCursor(in_table, final_fields , where_clause=where_clause)]
    fc_dataframe = pd.DataFrame(data, columns=final_fields)
    fc_dataframe = fc_dataframe.set_index(OIDFieldName, drop=True)
    return fc_dataframe


def check_float(element):
    # if element != ' ':
    partition = element.partition('.')

    if element.isdigit():
        newelement = round(float(element),2)
        # print('string is valid')

    elif (partition[0].isdigit() and partition[1] == '.' and partition[2].isdigit()) or (
            partition[0] == '' and partition[1] == '.' and partition[2].isdigit()) or (
            partition[0].isdigit() and partition[1] == '.' and partition[2] == ''):
        newelement = round(float(element),2)
        # print(newelement)
        # print("String is also valid !")
    else:
        newelement = np.NaN
        # print("string is not valid !")
    return  newelement
    # else:
    #     newelement = None
    #     return newelement
def calc_line_type(element):
    if element.isnull():
        line = 2
        print "arc"
    else:
        line = 1
        print "YASHAR"
    return line








input  = r"C:\Users\danielb\Documents\MAPI\BANKAL_MODERNI\INPROCESS\20220725\Lines_InProcess.gdb\Lines_InProcess"
output = r"C:\Users\danielb\Documents\MAPI\BANKAL_MODERNI\INPROCESS\RESUTLTS\MNCDB-TEMPLATE.gdb\InProcessFronts"

start = datetime.datetime.now()
print start
arcpy.Delete_management("in_memory")


##deleting the existing results data
print "DELETING"
arcpy.TruncateTable_management(output)

#converting the arcs feature class to pandas dataframe
table = table_to_data_frame(input, input_fields=None, where_clause=None) ## full
print datetime.datetime.now()
print "finished table"



##calculating the legal length in a new column without nulls or zeros
table['LEGAL'] = np.where(table['FRONT_LEGAL_LENGTH'].isnull() | table['FRONT_LEGAL_LENGTH'] ==0 , np.NaN, table['FRONT_LEGAL_LENGTH'])
print "finished Legal LENGTH"


##calculating the RADIUS in a new column without nulls or zeros
table['RADIUS'] = np.where(table['CUR_RADIUS'].isnull() | table['CUR_RADIUS'] ==0 , np.NaN, table['CUR_RADIUS'])
print "finished RADIUS"

##calculating the line type 1=staight 2 = arc
table['LINE_TYPE'] = np.where(table['CUR_RADIUS'].isnull() | table['CUR_RADIUS'] ==0 , 1, 2)
print datetime.datetime.now()
print "finished LINE TYPE"

# calulate the status of the line 1= old 2 = new
table['STATUS'] = np.NaN
table.loc[table['ENTITY_STATE'] == 0, 'STATUS'] = 1
table.loc[table['ENTITY_STATE'] == 1, 'STATUS'] = 2
print "finished LINE status"
##calculating uniqID
table.insert(0, 'UNIQUEid', range(1, 1 + len(table)))
print datetime.datetime.now()
print "finished incrementing"

print table.columns
print datetime.datetime.now()
print "starting upload"
#Tinserting the relevant features to the results only where STATUS in (1,3,6,16)
inputs = table.loc[0:,['SHAPE@JSON','STATUS','LINE_TYPE','RADIUS','LEGAL','UNIQUEid']].values.tolist()  ##'parcel_id_x' means parcel id left maybe change to 'parcel_id_y'
fields = ['SHAPE@JSON','LineStatus','LineType','Radius','LegalLength','FrontUniqueID']
InsertCursor = arcpy.da.InsertCursor(output, fields)
for row in inputs:
    try:
        InsertCursor.insertRow(row)
    except:
        pass


# result = table.head(10)
# print("First 10 rows of the DataFrame:")
# print(result)




end = datetime.datetime.now()
print end
print 20*'*'
print "***   FINISHED   ***"
print 20*'*'
print end-start